In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: system
    found: true
    include directory: unknown
    lib directory: unknown
    name: accelerate
    openblas configuration: unknown
    pc file directory: unknown
    version: unknown
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4428686672
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -falign-functions=8, -I/opt/homebrew/opt/libomp/include
    commands: cc
    linker: ld64
    linker args: -L/opt/homebrew/opt/libomp/lib, -falign-functions=8, -I/opt/homebrew/opt/libomp/include
    name: clang
    version: 16.0.0
  c++:
    args: -I/opt/homebrew/opt/libomp/include
    commands: c++
    linker: ld64
    linker args: -L/opt/homebrew/opt/libomp/lib, -I/opt/homebrew/opt/libomp/include
    name: clang
    version: 16.0.0
  cython:
    commands: cython
    linker: cython
    name: cython
   

# meta model

In [2]:
import pandas as pd

df_feature = pd.read_parquet("data/features_d002.parquet")
df_label = pd.read_parquet("data/label_d002.parquet")
df_label.head()

,close,ret,trgt,bin,side
index,,,,,
2020-01-01 00:00:00+00:00,7177.93,NaN,NaN,0.0,0.0
2020-01-01 00:03:00+00:00,7175.61,NaN,NaN,0.0,0.0
2020-01-01 00:06:00+00:00,7182.06,NaN,NaN,0.0,0.0
2020-01-01 00:09:00+00:00,7183.07,NaN,NaN,0.0,0.0
2020-01-01 00:12:00+00:00,7176.26,NaN,NaN,0.0,0.0


In [3]:
from custom_indicators.selection import META_1M, META_3M, META_15M

invalid_len = 300
date = "2024-09-01"
train_mask = (df_label.index <= date).tolist()
test_mask = (df_label.index > date).tolist()

meta_features_col = META_1M + META_3M + META_15M

train_feature = df_feature[train_mask][meta_features_col].iloc[invalid_len:]
train_label = df_label[train_mask].iloc[invalid_len:]
test_feature = df_feature[test_mask][meta_features_col].iloc[invalid_len:]
test_label = df_label[test_mask].iloc[invalid_len:]

print(train_feature.shape)
print(train_label.shape)
print(test_feature.shape)
print(test_label.shape)

valid_train = train_label["ret"].notna().tolist()
valid_test = test_label["ret"].notna().tolist()

train_feature = train_feature[valid_train]
train_label = train_label[valid_train]["bin"].astype(int)
test_feature = test_feature[valid_test]
test_label = test_label[valid_test]["bin"].astype(int)

print(train_feature.shape)
print(train_label.shape)
print(test_feature.shape)
print(test_label.shape)

test_label.value_counts()

(818101, 696)
(818101, 5)
(72659, 696)
(72659, 5)
(613257, 696)
(613257,)
(54596, 696)
(54596,)


bin
1    29437
0    25159
Name: count, dtype: int64

In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 50,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-6, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 1),
    }
    dtrain = lgb.Dataset(train_feature, train_label)
    dtest = lgb.Dataset(test_feature, test_label)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(test_feature)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(test_label, pred_label)
    recall = recall_score(test_label, pred_label)
    f1 = f1_score(test_label, pred_label)
    auc = roc_auc_score(test_label, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=50, show_progress_bar=True, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_87620/1575565261.py:41: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-18 21:11:15,430] A new study created in memory with name: no-name-4000899a-9aa4-4838-97f6-04c20060aef7
Best trial: 0. Best value: 0.68902:   2%|▏         | 1/50 [00:29<24:28, 29.98s/it]

precision: 0.6565762689605358, recall: 0.6793491184563645, f1: 0.6677685950413224
[I 2025-02-18 21:11:45,454] Trial 0 finished with value: 0.6890202492600234 and parameters: {'num_leaves': 81, 'max_depth': 85, 'min_gain_to_split': 0.7917527471138395, 'min_data_in_leaf': 198, 'lambda_l1': 0.8427185218316733, 'lambda_l2': 0.9585872286427635, 'num_boost_round': 964}. Best is trial 0 with value: 0.6890202492600234.


Best trial: 1. Best value: 0.68924:   4%|▍         | 2/50 [01:16<31:57, 39.96s/it]

precision: 0.6694607331726445, recall: 0.6321636036280871, f1: 0.6502778068979977
[I 2025-02-18 21:12:32,396] Trial 1 finished with value: 0.6892401713423447 and parameters: {'num_leaves': 164, 'max_depth': 36, 'min_gain_to_split': 0.5755777955102471, 'min_data_in_leaf': 84, 'lambda_l1': 0.540739611721992, 'lambda_l2': 0.1449106139036297, 'num_boost_round': 969}. Best is trial 1 with value: 0.6892401713423447.


Best trial: 2. Best value: 0.691876:   6%|▌         | 3/50 [02:11<36:27, 46.55s/it]

...

precision: 0.6545484186501467, recall: 0.6819648741379896, f1: 0.6679754437919044
[I 2025-02-18 21:42:32,430] Trial 47 finished with value: 0.6883716637296351 and parameters: {'num_leaves': 98, 'max_depth': 54, 'min_gain_to_split': 0.56464793270635, 'min_data_in_leaf': 147, 'lambda_l1': 0.34823848688575343, 'lambda_l2': 0.6816750476837917, 'num_boost_round': 942}. Best is trial 22 with value: 0.6936325706354514.


Best trial: 22. Best value: 0.693633:  98%|█████████▊| 49/50 [32:10<00:38, 38.48s/it]

precision: 0.6640681342079054, recall: 0.6568943846179978, f1: 0.6604617801762416
[I 2025-02-18 21:43:25,568] Trial 48 finished with value: 0.6897916201357639 and parameters: {'num_leaves': 252, 'max_depth': 71, 'min_gain_to_split': 0.21429477797519753, 'min_data_in_leaf': 60, 'lambda_l1': 0.2631006688269282, 'lambda_l2': 0.5177630673698519, 'num_boost_round': 855}. Best is trial 22 with value: 0.6936325706354514.


Best trial: 22. Best value: 0.693633: 100%|██████████| 50/50 [32:54<00:00, 39.50s/it]

precision: 0.665810214923093, recall: 0.6514250772836906, f1: 0.6585390981833167
[I 2025-02-18 21:44:10,291] Trial 49 finished with value: 0.6908376939466974 and parameters: {'num_leaves': 76, 'max_depth': 75, 'min_gain_to_split': 0.1464913951589302, 'min_data_in_leaf': 108, 'lambda_l1': 0.5922931689963664, 'lambda_l2': 0.6541485525643781, 'num_boost_round': 773}. Best is trial 22 with value: 0.6936325706354514.


In [8]:
model = lgb.train(
    study.best_params,
    lgb.Dataset(train_feature, train_label),
)

In [15]:
model.save_model("custom_indicators/model_meta.txt")

# side models

In [17]:
from custom_indicators.selection import SIDE_1M, SIDE_3M, SIDE_15M

invalid_len = 300
date = "2024-09-01"
train_mask = (df_label.index <= date).tolist()
test_mask = (df_label.index > date).tolist()

side_features_col = SIDE_1M + SIDE_3M + SIDE_15M

train_feature = df_feature[train_mask][side_features_col].iloc[invalid_len:]
train_label = df_label[train_mask].iloc[invalid_len:]
test_feature = df_feature[test_mask][side_features_col].iloc[invalid_len:]
test_label = df_label[test_mask].iloc[invalid_len:]

print(train_feature.shape)
print(train_label.shape)
print(test_feature.shape)
print(test_label.shape)

valid_train = (train_label["side"].astype(int) != 0).tolist()
valid_test = (test_label["side"].astype(int) != 0).tolist()

train_feature = train_feature[valid_train]
train_label = (train_label[valid_train]["side"].astype(int) == 1).astype(int)
test_feature = test_feature[valid_test]
test_label = (test_label[valid_test]["side"].astype(int) == 1).astype(int)

print(train_feature.shape)
print(train_label.shape)
print(test_feature.shape)
print(test_label.shape)

test_label.value_counts()

(818101, 374)
(818101, 5)
(72659, 374)
(72659, 5)
(318731, 374)
(318731,)
(29437, 374)
(29437,)


side
1    14791
0    14646
Name: count, dtype: int64

In [18]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 50,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-6, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 1),
    }
    dtrain = lgb.Dataset(train_feature, train_label)
    dtest = lgb.Dataset(test_feature, test_label)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
    )
    pred_proba = model.predict(test_feature)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(test_label, pred_label)
    recall = recall_score(test_label, pred_label)
    f1 = f1_score(test_label, pred_label)
    auc = roc_auc_score(test_label, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=50, show_progress_bar=True, n_jobs=1)

/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_87620/1575565261.py:41: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-18 21:51:27,671] A new study created in memory with name: no-name-572793ea-48b0-4a09-b24d-8d5342580601
Best trial: 0. Best value: 0.841762:   2%|▏         | 1/50 [00:36<29:34, 36.22s/it]

precision: 0.7506438931815101, recall: 0.7487661415725779, f1: 0.7497038415975631
[I 2025-02-18 21:52:03,891] Trial 0 finished with value: 0.8417623623091695 and parameters: {'num_leaves': 293, 'max_depth': 75, 'min_gain_to_split': 0.4092636149889744, 'min_data_in_leaf': 152, 'lambda_l1': 0.39451378542392707, 'lambda_l2': 0.37463037797800447, 'num_boost_round': 947}. Best is trial 0 with value: 0.8417623623091695.


Best trial: 0. Best value: 0.841762:   4%|▍         | 2/50 [00:44<15:56, 19.93s/it]

precision: 0.7451400992090093, recall: 0.751538097491718, f1: 0.748325423272409
[I 2025-02-18 21:52:12,411] Trial 1 finished with value: 0.8387139198445032 and parameters: {'num_leaves': 47, 'max_depth': 42, 'min_gain_to_split': 0.05279199805084433, 'min_data_in_leaf': 32, 'lambda_l1': 0.7578423457997577, 'lambda_l2': 0.690214657899998, 'num_boost_round': 101}. Best is trial 0 with value: 0.8417623623091695.


Best trial: 2. Best value: 0.842295:   6%|▌         | 3/50 [01:11<17:57, 22.93s/it]

...

precision: 0.7490601503759399, recall: 0.7543776620918126, f1: 0.7517095024758311
[I 2025-02-18 22:08:26,443] Trial 47 finished with value: 0.8428460169222229 and parameters: {'num_leaves': 47, 'max_depth': 58, 'min_gain_to_split': 0.5067688800989126, 'min_data_in_leaf': 143, 'lambda_l1': 0.7060529480388211, 'lambda_l2': 0.45950842370077843, 'num_boost_round': 772}. Best is trial 12 with value: 0.8439344031273821.


Best trial: 12. Best value: 0.843934:  98%|█████████▊| 49/50 [17:21<00:20, 20.05s/it]

precision: 0.7482357685328315, recall: 0.7526874450679467, f1: 0.7504550050556117
[I 2025-02-18 22:08:49,211] Trial 48 finished with value: 0.8425025633457934 and parameters: {'num_leaves': 79, 'max_depth': 72, 'min_gain_to_split': 0.077197073387469, 'min_data_in_leaf': 114, 'lambda_l1': 0.4722545650164571, 'lambda_l2': 0.7356334728983056, 'num_boost_round': 911}. Best is trial 12 with value: 0.8439344031273821.


Best trial: 12. Best value: 0.843934: 100%|██████████| 50/50 [17:43<00:00, 21.28s/it]

precision: 0.7501183471968621, recall: 0.7499154891488067, f1: 0.7500169044560147
[I 2025-02-18 22:09:11,492] Trial 49 finished with value: 0.8427275332397115 and parameters: {'num_leaves': 107, 'max_depth': 54, 'min_gain_to_split': 0.17656236357918503, 'min_data_in_leaf': 197, 'lambda_l1': 0.5471249252686011, 'lambda_l2': 0.1338520268095339, 'num_boost_round': 470}. Best is trial 12 with value: 0.8439344031273821.


In [19]:
model = lgb.train(
    study.best_params,
    lgb.Dataset(train_feature, train_label),
)

In [20]:
model.save_model("custom_indicators/model_side.txt")